## The great 1 dimensional bandstructure function
 **Autors:** *Klára Nováková* and *Andris Potrebko* with a vital teoretical support of *Andreas Wacker*, *Tor Sjöstrand* and *Hofmanns* book *Solid State Physics*
 
**Year:** *2019*

* If you are looking at this file from the universities computer which is used for the lab, then please leave everything exactly the way it was before.

* The file is structured to be able to run cell by cell. If any of the variable is unclear, it is possible to create a new cell by pressing *Esc* and then *A* and play around. The shortcut to delete cell: *Esc* and then *D+D*.

* In the following document, the one dimensional bandstructure of a crystal is solved using the Plane wave basis approximation (further described in the lab manual, here only a summary of presented, stating main variables).

* In this case the atomic potential is approximated by the Gaussian potential $V(x)=-U\sum_n e^{-(x-na)^2/2\sigma^2}$. This potential can be written in the Fourier components as $V(x)=\sum_{G_j}-U\sqrt {2\pi}\frac{\sigma}{a}e^{-G^2_j \sigma^2/2} e^{i G_j x} \equiv  \sum_{G_j} V_{G_j} \cdot e^{i G_j x}$,
where $G_j$ is a reciprocal lattice vector.

* The ampliture of $V_G$, we denote as $A$, that is $A \equiv -U\sqrt {2\pi}\frac{\sigma}{a}$ and  $V_G = A e^{-G^2 \sigma^2/2} $.

* In order to obtain the energy spectrum or the band structure we need to solve the following eigenvalue equation:
$\Big( \frac{\hbar^2 (k+K)^2}{2m} - \varepsilon \Big) c_{k+K} +\sum_G c_{k+K-G} V_G$ and search for $\varepsilon$ - the energy eigenvalues.


### Necessary imports

In [1]:
# Usual imports
import matplotlib.pyplot as plt
%matplotlib notebook 
import numpy as np

# pretty print all cell's output and not just the last one
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Widgets imports
import ipywidgets as widgets
from ipywidgets import interact, interactive
from IPython.display import display
from ipywidgets import HBox, Label
from ipywidgets import IntSlider

There are many variables involving *k* but it is important that you note the differences;
* *K* the reciprocal lattice vector and *k* the wave vector $k\in [-\pi/a; \pi/a]$ 
* The number of *k* vectors in our lattice is *Len_k*
* *MaxK* is the maximal reciprocal lattice vector we consider to include in the sum ($K$ or $K-G$) thus the summation goes from *-MaxK* to *MaxK*.

### Definition of initial values

In [ ]:
a = 3  # Latice constant in Åmstrongs
sigma = 0.5  # Very arbitrary value
U = 11  # eV
maxK = 6
Len_k = 15  # How fine to calculate the band structure
NrOfK = maxK*2+1
A = -U*np.sqrt(2 * np.pi)*sigma/a

*BandStructureFunction.ipynb* file contains a function *BandStructF* which creates the matrix for the eigenvalue equation described above and solves for the eigenvalues - energies.
*%run* is one of the so-called magic functions and does the import of the function found in the file *BandStructureFunction.ipynb*.

In [ ]:
%run BandStructureFunction.ipynb

Solve eigenvalue equation for each wave number *k*
* Put k vector into right magnitude.  Before *k* is some integer $k =.. -2,-1,0,1..$. Save the obtained value in *kVect*
* Save calculated energies in *Energies*
* Uncomment "%%time" to measure the calculation time of the cell

In [ ]:
# %%time
Energies = np.zeros(shape=(Len_k*2+1, NrOfK))
# Vector for plotting containing all of the k values in 1/Angstrom
kVect = [0]*(Len_k*2+1)
for ki in range(-Len_k, Len_k+1):  # k vector for which we are calculating
    k = np.pi/(a)*ki/Len_k
#     %run BandStructureFunction.ipynb
    E = BandStructF()
    Energies[ki+Len_k] = np.real(E)
    kVect[ki+Len_k] = k  # in 1/Angstrom

## Plots

### Plot potential

Plots are hidden so that they would not appear when running the interface. To make the plots visible either comment out the lines containg *%%capture* or comment out the cell containing *fig*.

In [ ]:
%%capture
from matplotlib import pyplot as plt
# If you want to display the figure here, then uncomment previous line
%matplotlib notebook
%matplotlib notebook
fig, [ax, ax2] = plt.subplots(nrows=1, ncols=2)  # Two plots in the same figure
fig.set_size_inches(9.5, 3.5)
fig.subplots_adjust(wspace=0.4, bottom=0.2)  # Margins around the subplots


x = np.linspace(-5*a, 5*a, 401)
# length of the x vector should be an odd number so that 0 is included,
# otherwise it does not plot the peak correctly
Vx = -U*np.exp(-x**2/(2*sigma**2))
VxSum = [0]*len(x)
for n in range(-5, 6):
    Vx1 = -U*np.exp(-(x-n*a)**2/(2*sigma**2))
    ax.plot(x, Vx1, 'y--')
    VxSum = VxSum+Vx1
ax.plot(x, VxSum)
ax.plot(x, Vx)
ax.set(xlabel='a, ($\AA$)', ylabel='V (eV)', title='Atomic potential')

### Plot energies

In [ ]:
%%capture
ax2.plot(kVect, Energies[:, 0:5], color='purple')
ax2.set(xlabel='k ($1/ \AA$)', ylabel='E (eV)', title='Band structure')

In [ ]:
# fig

### Creating an interactive user interface

Function *f* repeats the steps above whenever some of the widgets are going to be used. Function *interactive* creates these widgets and connects them to the function *f*.

In [ ]:
def f(Lattice_Constant=a, Amplitude=U, NumKVec=maxK, PlotK=5):
    # So that BandStructureFunction.ipynb knows the variables used in the function f
    global a, A, k, Len_k, maxK, U, NrOfK
    a = Lattice_Constant
    U = Amplitude
    maxK = NumKVec
    A = -U*np.sqrt(2 * np.pi)*sigma/a  # np.sqrt(2*sigma)
    NrOfK = maxK*2+1
    Energies = np.zeros(shape=(Len_k*2+1, NrOfK))
    kVect = [0]*(Len_k*2+1)
    for ki in range(-Len_k, Len_k+1):
        k = np.pi/(a)*ki/Len_k
        E = BandStructF()
#         %run BandStructureFunction.ipynb
        Energies[ki+Len_k] = np.real(E)
        kVect[ki+Len_k] = k
    ax.cla()
    ax.set(xlabel='a, ($\AA$)', ylabel='V (eV)', title='Atomic potential')
    x = np.linspace(-5*a, 5*a, 401)
    Vx = -U*np.exp(-x**2/(2*sigma**2))
    VxSum = [0]*len(x)
    for n in range(-5, 6):
        Vx1 = -U*np.exp(-(x-n*a)**2/(2*sigma**2))
        ax.plot(x, Vx1, 'y--')
        VxSum = VxSum+Vx1

    ax.plot(x, VxSum)
    ax.plot(x, Vx)

    ax2.cla()
    ax2.set(xlabel='k ($1/ \AA$)', ylabel='E (eV)', title='Band structure')
    ax2.plot(kVect, Energies[:, 0:PlotK], color='purple')

    # return Energies


The_Interaction = interactive(f, Lattice_Constant=(1, 10, 0.5), Amplitude=(
    0, 100, 1), NumKVec=(1, 20, 1), PlotK=(0, 10, 1))

### Creating boxes for widgets that are to be displayed

In [ ]:
for widg in The_Interaction.children[:-1]:
    widg.description = ""
    widg.continuous_update = False


Lattice_Const, Potential_Amp, Num_Of_K, PlotK = [
    The_Interaction.children[i] for i in range(4)]

# display(Lattice_Const,Potential_Amp ,Num_Of_K ) #by uncommenting the widgets will be displayed here
FirstBox = widgets.HBox([Label(r'Lattice constant (Å)'), Lattice_Const, Label(
    r'Potential depth, (eV)'), Potential_Amp, ])
SecondBox = widgets.HBox(
    [Label(r'N of K vectors'), Num_Of_K, Label(r'How many bands to plot'), PlotK])

The number of calculated bands depends on *maxK* as $2\cdot maxK + 1$. So we set the maximal possible band to be plotted accordingly.

In [ ]:
def PlotKMax(*args):
    PlotK.max = 2*Num_Of_K.value+1


Num_Of_K.observe(PlotKMax, 'value')